In [10]:
import pandas as pd

In [33]:
dataamazon = pd.read_csv( "datasets/amazon_prime_titles.csv")
datadisney = pd.read_csv( "datasets/disney_plus_titles.csv")
datahulu = pd.read_csv( "datasets/hulu_titles.csv")
datanetflix = pd.read_csv("datasets/netflix_titles.csv")


#Data transform, add score

data1 = pd.read_csv("datasets/ratings/1.csv")#,usecols=(1,3))
data2 = pd.read_csv("datasets/ratings/2.csv")#, usecols=(1,3))
data3 = pd.read_csv("datasets/ratings/3.csv")#, usecols=(1,3))
data4 = pd.read_csv("datasets/ratings/4.csv")#, usecols=(1,3))
data5 = pd.read_csv("datasets/ratings/5.csv")#, usecols=(1,3))
data6 = pd.read_csv("datasets/ratings/6.csv")#, usecols=(1,3))
data7 = pd.read_csv("datasets/ratings/7.csv")#, usecols=(1,3))
data8 = pd.read_csv("datasets/ratings/8.csv")#, usecols=(1,3))


#drop duplicates
data1.drop_duplicates(inplace=True)
data2.drop_duplicates(inplace=True)
data3.drop_duplicates(inplace=True)
data4.drop_duplicates(inplace=True)
data5.drop_duplicates(inplace=True)
data6.drop_duplicates(inplace=True)
data7.drop_duplicates(inplace=True)
data8.drop_duplicates(inplace=True)

In [12]:
# MAKE COLUMNS ID = FIRST LETTER + SHOW_ID
dataamazon["ID"] = "a"+ dataamazon["show_id"]
datadisney["ID"] = "d"+ datadisney["show_id"]
datahulu["ID"] = "h"+ datahulu["show_id"]
datanetflix["ID"] = "n"+ datanetflix["show_id"]

# DROP COLUMN SHOW_ID
dataamazon.drop(columns = ["show_id"], inplace = True)
datadisney.drop(columns = ["show_id"], inplace = True)
datahulu.drop(columns = ["show_id"], inplace = True)
datanetflix.drop(columns = ["show_id"], inplace = True)



In [13]:
# concatenate df
datatotal2 = pd.concat([dataamazon,datanetflix,datahulu,datadisney], ignore_index= True)

#rename column for merge
datatotal2.rename(columns={'ID':'movieId'},inplace=True)

#concatenate rating

datarating = pd.concat([data1,data2,data3,data4,data5,data6,data7,data8], ignore_index= True)

#drop column that I am not going to use
datarating.drop(columns = ["userId","timestamp"], inplace = True)

#group by movieId + mean

datarprom = datarating.groupby(['movieId']).mean()

#Merge

datatotal = pd.merge(datatotal2,datarprom, on=["movieId"])


In [14]:
#RENAME COLUMNS,TO PREVENT CHANGES IN INITIAL DATAFRAME

datatotal.rename(columns={'movieId':'ID'},inplace=True)
datatotal.rename(columns={'rating_x':'rating'},inplace=True)
datatotal.rename(columns={'rating_y':'score'},inplace=True)
datatotal.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,ID,score
0,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,as1,3.467131
1,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,as2,3.548682
2,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,as3,3.500000
3,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",as4,3.538055
4,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,as5,3.478992


In [15]:
#The null values of the rating field must be replaced by the string “G”
datatotal["rating"].fillna("G", inplace = True)

#If there are dates, they must have the format YYYY-mm-dd

datatotal["date_added"] = pd.to_datetime(datatotal["date_added"])


#Text fields must be in lowercase, no exceptions
datatotal["type"] = datatotal["type"].str.lower()
datatotal["title"] = datatotal["title"].str.lower()
datatotal["director"] = datatotal["director"].str.lower()
datatotal["cast"] = datatotal["cast"].str.lower()
datatotal["country"] = datatotal["country"].str.lower()
datatotal["rating"] = datatotal["rating"].str.lower()
datatotal["listed_in"] = datatotal["type"].str.lower()
datatotal["description"] = datatotal["description"].str.lower()
datatotal["ID"] = datatotal["ID"].str.lower()
datatotal["duration"] = datatotal["duration"].str.lower()

In [16]:
#The field duration must be converted to two fields: duration_int and duration_type. 
#The first will be an integer and the second a string indicating the unit of duration measurement: min (minutes) or season (seasons).
datatotal[['duration_int','duration_type']]=datatotal.duration.str.split(' ',expand=True)

#Drop tabla duration

datatotal.drop(columns = ["duration"], inplace = True)


In [17]:
#ETL, REPLACE SEASONS TO SEASON for EDA
datatotal["duration_type"].replace("seasons","season", inplace= True)

In [18]:
#save
datatotal.to_csv("datasets/datatotal.csv", sep=',', index= False)


In [19]:
datatotal["ID"].to_csv("datasets/Id.csv", sep=',')

In [20]:
datatotal.head()

,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,ID,score,duration_int,duration_type
0,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,movie,a small fishing village must procure a local d...,as1,3.467131,113,min
1,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,movie,a metro family decides to fight a cyber crimin...,as2,3.548682,110,min
2,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,movie,after a man discovers his wife is cheating on ...,as3,3.500000,74,min
3,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,movie,"pink breaks the mold once again, bringing her ...",as4,3.538055,69,min
4,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,movie,teenage matt banting wants to work with a famo...,as5,3.478992,45,min


In [21]:
#Movie with longer duration with optional filters of YEAR, PLATFORM AND TYPE OF DURATION. (the function should be called get_max_duration(year, platform, duration_type))

def get_max_duration(year, platform, duration_type):
    lista=[]
    lista2=[]
    for ind,z in enumerate (datatotal["release_year"]):
        if (z == year):
            if (datatotal["duration_type"][ind] == duration_type):
                if datatotal["ID"][ind][0] == platform:
                    lista.append(int(datatotal["duration_int"][ind]))
                    lista2.append(ind)
                    b = max(lista)
                    c = lista.index(b)
                    d = lista2[c]

    return (datatotal["title"][d])
                



In [22]:
get_max_duration(2004,"h","min")

'the manchurian candidate'

In [23]:
datatotal.iloc[1673,:]

type                                                         movie
title                      midnight thunderstorm for sleep 9 hours
director                                               mark knight
cast                                                           NaN
country                                                        NaN
date_added                                                     NaT
release_year                                                  2014
rating                                                         all
listed_in                                                    movie
description      our most popular ambient sleep video is a full...
ID                                                          as1674
score                                                      3.50303
duration_int                                                   541
duration_type                                                  min
Name: 1673, dtype: object

In [24]:
#Number of movies per platform with a score greater than XX in a given year (the function must be called get_score_count(platform, scored, year))
def get_score_count(platform, scored, year):
    lista=[]
    for ind,z in enumerate (datatotal["release_year"]):
        if (z == year):
            if datatotal["ID"][ind][0] == platform:
                if datatotal["score"][ind] > scored:
                     lista.append(float(datatotal["score"][ind]))
                
    return len(lista)

In [25]:
get_score_count("a",2.5,2020)

962

In [26]:
#Number of movies per platform with PLATFORM filter. (The function should be called get_count_platform(platform))
def get_count_platform(platform):
    a=0
    for z in (datatotal["ID"]):
        if z[0] == platform:
            a += 1
    return a

In [27]:
get_count_platform("h")

3073

In [28]:
#Actor who repeats himself the most according to platform and year. (The function should be called get_actor(platform, year))

def get_actor(platform, year):
    lista=[]
    lista4 =[]
    b=0
    if platform == "h":
        return("Hulu no tiene base de datos de actores")
    for ind,z in enumerate (datatotal["release_year"]):
        if (z == year):
            if datatotal["ID"][ind][0] == platform:
                lista.append(datatotal["cast"][ind])  #ARMO LISTA EN BASE A PLATAFORMA Y AÑO
    for indice, a in enumerate (lista): #SUBDIVIDO LA LISTA POR CADA NOMBRE
     if type(a)== str:
          lista[indice]= a.split(", ")

    for a in lista:
        if type(a)== list: # ARMO UNA LISTA APARTE SIN LOS NAN
            lista4.append(a)
    flatlist =[elemento for sublista in lista4 for elemento in sublista]  #FLAT
    for ind,a in enumerate(flatlist):  #CON COUNT BUSCO CUAL SE REPITE MAS Y LO RETORNO
        if flatlist.count(a) > b:
            b= flatlist.count(a)
            d= flatlist[ind]
    return(d)

In [29]:
get_actor("a",2014)

'anne-marie newland'

In [30]:
film_time = []
film_name = []
for k, l in enumerate (datatotal['release_year']):
    if l == 2016:
        if datatotal['ID'][k][0] == "h":
            if datatotal['duration_type'][k] == "min":
                film_time.append(int(datatotal['duration_int'][k]))
                film_name.append(k)
                maximo = max(film_time)
                film = film_time.index(maximo)
                show_name = datatotal['title'][film_name[film]]
print(show_name)

silence
